# Context-FID Score Presentation
## Necessary packages and functions call

- Context-FID score: A useful metric measures how well the the synthetic time series windows ”fit” into the local context of the time series

In [1]:
import os
import torch
import numpy as np
import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))
from Utils.context_fid import Context_FID
from Utils.metric_utils import display_scores
from Utils.cross_correlation import CrossCorrelLoss

## Data Loading

Load original dataset and preprocess the loaded data.

In [2]:
# iterations = 5
# ori_data = np.load('../toy_exp/samples/sine_ground_truth_24_train.npy')
# # ori_data = np.load('../OUTPUT/{dataset_name}/samples/{dataset_name}_norm_truth_{seq_length}_train.npy')  # Uncomment the line if dataset other than Sine is used.
# fake_data = np.load('../toy_exp/ddpm_fake_sines.npy')


iterations = 5
# ori_data = np.load('../toy_exp/samples/sine_ground_truth_24_train.npy')
ori_data = np.load('../OUTPUT/test/samples/morning_peak_etth_norm_truth_24_train.npy')  # Uncomment the line if dataset other than Sine is used.
fake_data = np.load('../OUTPUT/test/ddpm_fake_morning_peak_etth_milestone_1000.npy')

In [3]:
print('ori shape is: ', ori_data.shape)
print('fake shape is: ', fake_data.shape)

ori shape is:  (2881, 24, 7)
fake shape is:  (3600, 24, 7)


## Context-FID Score

- The Frechet Inception distance-like score is based on unsupervised time series embeddings. It is able to score the fit of the fixed length synthetic samples into their context of (often much longer) true time series.

- The lowest scoring models correspond to the best performing models in downstream tasks

In [6]:
for j in range(10):

    context_fid_score = []

    for i in range(iterations):
        context_fid = Context_FID(ori_data[:], fake_data[:ori_data.shape[0]])
        context_fid_score.append(context_fid)
        print(f'Iter {i}: ', 'context-fid =', context_fid, '\n')

    display_scores(context_fid_score)

# single mp input, 360epoch,   0.354



/mnt/data728/duyin/anaconda3/envs/torch201/lib/python3.9/site-packages/torch/nn/modules/conv.py:309: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343997789/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,


Iter 0:  context-fid = 0.25906747324115403 

Iter 1:  context-fid = 0.2933629537594794 

Iter 2:  context-fid = 0.28260160660836453 

Iter 3:  context-fid = 0.29062610840063285 

Iter 4:  context-fid = 0.28430485379894793 

Final Score:  0.2819925991617157 ± 0.016832463275340992
Iter 0:  context-fid = 0.30897270475322347 

Iter 1:  context-fid = 0.26985096079599513 

Iter 2:  context-fid = 0.2894612259710446 

Iter 3:  context-fid = 0.2769913049817549 

Iter 4:  context-fid = 0.2654812027399775 

Final Score:  0.28215147984839917 ± 0.02176136871458309
Iter 0:  context-fid = 0.2719209582662512 

Iter 1:  context-fid = 0.29380294800932927 

Iter 2:  context-fid = 0.28245384941378127 

Iter 3:  context-fid = 0.2716057679803626 

Iter 4:  context-fid = 0.26794041372508337 

Final Score:  0.27754478747896155 ± 0.01313422991756374
Iter 0:  context-fid = 0.3119169718703485 

Iter 1:  context-fid = 0.2526349257152799 

Iter 2:  context-fid = 0.29707991480082574 

Iter 3:  context-fid = 0.30534

## Correlational Score

- The metric uses the absolute error of the auto-correlation estimator by real data and synthetic data as the metric to assess the temporal dependency.

- For d > 1, it uses the l1-norm of the difference between cross correlation matrices.

In [7]:
def random_choice(size, num_select=100):
    select_idx = np.random.randint(low=0, high=size, size=(num_select,))
    return select_idx

In [ ]:
x_real = torch.from_numpy(ori_data)
x_fake = torch.from_numpy(fake_data)

correlational_score = []
size = int(x_real.shape[0] / iterations)

for i in range(iterations):
    real_idx = random_choice(x_real.shape[0], size)
    fake_idx = random_choice(x_fake.shape[0], size)
    corr = CrossCorrelLoss(x_real[real_idx, :, :], name='CrossCorrelLoss')
    loss = corr.compute(x_fake[fake_idx, :, :])
    correlational_score.append(loss.item())
    print(f'Iter {i}: ', 'cross-correlation =', loss.item(), '\n')

display_scores(correlational_score)

# double input, 0.181
# single input  mp, 360 epoch,    0.084 + 0.02

Iter 0:  cross-correlation = 0.11397961049790839 

Iter 1:  cross-correlation = 0.11904152512068163 

Iter 2:  cross-correlation = 0.04847594380661483 

Iter 3:  cross-correlation = 0.09386066591843291 

Iter 4:  cross-correlation = 0.09537706316327271 

Final Score:  0.09414696170138209 ± 0.03456930737114451


: 